In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random as rn
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
path =   # Enter the directory of your project folder
PATH = path + "/Results"

if not os.path.exists(PATH):
    os.mkdir(PATH) 

In [ ]:
df = pd.read_csv(path+"//ToyDataset.csv").drop(['Unnamed: 0'],axis=1)
df1 = pd.read_csv(PATH+"//Data one hot encoded.csv").drop(['Unnamed: 0'],axis=1)
clientID = df.ClientID.unique()

df["EntryDate"] = pd.to_datetime(df['EntryDate'])
df["ExitDate"] = pd.to_datetime(df['ExitDate'])
df = df.sort_values(["EntryDate",'ExitDate']).reset_index(drop=True)
df1["EntryDate"] = pd.to_datetime(df1['EntryDate'])
df1["ExitDate"] = pd.to_datetime(df1['ExitDate'])
df1 = df1.sort_values(["EntryDate",'ExitDate']).reset_index(drop=True)

In [ ]:
# separating the living situation and the exit destination into categories
goal = [10,11]
closer = [3,19,20,21,22,23,25,26,28,31]
trans = [1,2,4,5,6,12,13,14,15,18,27,29]
no_progress = [7,16]
hard_to_judge = [8,9,17,24,30,99]

In [ ]:
# preparing the data for target
T = pd.DataFrame()
t = ["ultimate goal","closer to exit","transitional phase","no progress or worse","hard to judge"]
for i in [goal,closer,trans,no_progress,hard_to_judge]:
    
    temp = df1.loc[df[df['Destination'].isin(i)].index] 
    temp = temp.drop(['EnrollmentID', 'ClientID', 'ProjectID', 'EntryDate', 
                'ProjectType', 'ExitID', 'ExitDate', 'HouseholdID'],axis = 1)
    if i == goal:
        y = pd.DataFrame(["ultimate goal"],index=['target'])
    elif i == closer:
        y = pd.DataFrame(["closer to exit"],index=['target'])
    elif i == trans:
        y = pd.DataFrame(["transitional phase"],index=['target'])
    elif i == no_progress:
        y = pd.DataFrame(["no progress or worse"],index=['target'])
    elif i == hard_to_judge:
        y = pd.DataFrame(["hard to judge"],index=['target'])
        
    x = pd.concat([y,temp.sum(axis=0)])
    T = T.append(x.T,ignore_index=True)
    
# preparing the data for project type
A = df1[["EnrollmentID","ClientID","ProjectType"]]
M = df1.drop(["ProjectType"],axis = 1)
p = np.sort(A['ProjectType'].unique())
P = len(p)
F = pd.DataFrame()
for i in range(P):
    S = A[A["ProjectType"]==p[i]]
    M_S = M[M['EnrollmentID'].isin(S['EnrollmentID'])].drop(['EnrollmentID', 'ClientID', 'ProjectID', 'EntryDate',
        'ExitID','ExitDate','HouseholdID'],axis = 1)
    y = pd.DataFrame([p[i]],index=['target'])
    M_S = pd.concat([y,M_S.sum(axis=0)])
    F = F.append(M_S.T,ignore_index=True)

F = F.append(T,ignore_index = True)
F = F.set_index('target')

In [ ]:


# calculating tf_idf for project type
tf_idf = pd.DataFrame([],index = F.index, columns = F.columns)
for i in range(len(F.index)):
    for j in range(len(F.columns)):
        temp = F[F.columns[j]]
        tf = temp.loc[F.index[i]]
        n = len(temp[temp!=0])+1
        tf_idf.loc[F.index[i]][F.columns[j]] = tf * max(0,np.log(P/n))

tf_idf_norm = pd.DataFrame([],index = tf_idf.index, columns = tf_idf.columns)
total = list(p) + t  
for i in  total:
    if np.sqrt(np.square(tf_idf).sum(axis=1)).loc[i] == 0:
            tf_idf_norm.loc[i] = 0
    else:
        tf_idf_norm.loc[i] = tf_idf.loc[i]/np.sqrt(np.square(tf_idf).sum(axis=1)).loc[i]
    
tf_idf_norm.to_csv(PATH+"//Normalized TF_IDF for project type and target.csv")

tf_idf_norm = pd.read_csv(PATH+"//Normalized TF_IDF for project type and target.csv")
tf_idf_norm = tf_idf_norm.set_index("target",drop=True)

In [ ]:
data = pd.DataFrame(columns=['ClientID','click',"dest","backlink",'cos-tf-idf'])
for i in range(len(clientID)):
    temp = df[df['ClientID']==clientID[i]].reset_index(drop=True)
    closed = [temp['ProjectType'].iloc[0]]
    backlink = 0
    for k in range(len(temp)-1):
        cos = pd.DataFrame(tf_idf_norm.loc[str(temp['ProjectType'].iloc[k])]) 
        cos = cos.T.append(tf_idf_norm.loc[str(temp['ProjectType'].iloc[k+1])],ignore_index=True)
        cosine_sim = cosine_similarity(cos)[0][1]
        if temp['ProjectType'].iloc[k+1] not in closed:
            closed = closed + [temp['ProjectType'].iloc[k+1]]
            backlink = 0
        else:
            backlink = 1
            
        if temp['Destination'].iloc[-1] in goal:
            if backlink == 1:
                data = data.append({'ClientID':clientID[i],'click':(temp['ProjectType'].iloc[k],temp['ProjectType'].iloc[k+1]),"dest":'successful','cos-tf-idf':cosine_sim,'backlink':'yes'},ignore_index=True)
            elif backlink == 0:
                data = data.append({'ClientID':clientID[i],'click':(temp['ProjectType'].iloc[k],temp['ProjectType'].iloc[k+1]),"dest":'successful','cos-tf-idf':cosine_sim,'backlink':'no'},ignore_index=True)
        elif np.isnan(temp['Destination'].iloc[-1]):
            if backlink == 1:
                data = data.append({'ClientID':clientID[i],'click':(temp['ProjectType'].iloc[k],temp['ProjectType'].iloc[k+1]),"dest":'abandoned','cos-tf-idf':cosine_sim,'backlink':'yes'},ignore_index=True)
            elif backlink == 0:
                data = data.append({'ClientID':clientID[i],'click':(temp['ProjectType'].iloc[k],temp['ProjectType'].iloc[k+1]),"dest":'abandoned','cos-tf-idf':cosine_sim,'backlink':'no'},ignore_index=True)        
        else:
            if backlink == 1:
                data = data.append({'ClientID':clientID[i],'click':(temp['ProjectType'].iloc[k],temp['ProjectType'].iloc[k+1]),"dest":'unsuccessful','cos-tf-idf':cosine_sim,'backlink':'yes'},ignore_index=True)
            elif backlink == 0:
                data = data.append({'ClientID':clientID[i],'click':(temp['ProjectType'].iloc[k],temp['ProjectType'].iloc[k+1]),"dest":'unsuccessful','cos-tf-idf':cosine_sim,'backlink':'no'},ignore_index=True)


            

In [ ]:
change = pd.DataFrame(columns=['ClientID','click','backlink','change','type','dest'])
for i in range(len(clientID)):
    temp = data[data['ClientID']==clientID[i]].reset_index(drop=True)
    if len(temp) > 1:
        for k in range(1,len(temp)):
            c = temp['cos-tf-idf'].iloc[k]-temp['cos-tf-idf'].iloc[k-1]
            if c > 0:
                change = change.append({'ClientID':clientID[i],'click':temp['click'].iloc[k],'backlink':temp['backlink'].iloc[k],'change':c,'type':'increase','dest':temp['dest'].iloc[k]},ignore_index=True)
            elif c == 0:
                change = change.append({'ClientID':clientID[i],'click':temp['click'].iloc[k],'backlink':temp['backlink'].iloc[k],'change':c,'type':'no change','dest':temp['dest'].iloc[k]},ignore_index=True)
            elif c < 0:
                change = change.append({'ClientID':clientID[i],'click':temp['click'].iloc[k],'backlink':temp['backlink'].iloc[k],'change':c,'type':'decrease','dest':temp['dest'].iloc[k]},ignore_index=True)


In [ ]:
change_3var = change.groupby(['dest','type','backlink']).count()['ClientID']

In [ ]:
change_2var = change.groupby(['dest','type']).count()['ClientID']

In [ ]:
dest_unique = change.dest.unique()
type_unique = change.type.unique()
backlink_unique = change.backlink.unique()

In [ ]:
prob = pd.DataFrame(columns=['dest','conditioned on','backlink','prob'])
for i in dest_unique:
    var3 = change_3var.loc[i]
    var2 = change_2var.loc[i]
    for j in type_unique:
        total = var2.loc[j]
        prob_cond = var3.loc[j]/total
        for k in backlink_unique:
            if k not in prob_cond:
                continue
            prob = prob.append({'dest':i,'backlink':k,'conditioned on':j,'prob':prob_cond.loc[k]},ignore_index=True)
    

In [ ]:
prob